## Downloading Data

In [7]:
import zipfile
import os
from kaggle.api.kaggle_api_extended import KaggleApi


def get_kaggle_data_set(
    project: str, 
    document: str, 
    directory_to_extract_to: str
    ):
    api = KaggleApi()
    api.authenticate()

    api.dataset_download_file(project, document)
    zip_document = document + ".zip"
    with zipfile.ZipFile(zip_document, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

    os.remove(zip_document)


document = "cyberbullying_tweets.csv"
project = "andrewmvd/cyberbullying-classification"
directory_to_extract_to = r"C:\Users\EMILIO\Documents\Python Scripts\CyberbullingProject\model\data"

get_kaggle_data_set(project, document, directory_to_extract_to)
